In [2]:
import pandas as pd

In [3]:
json_data = pd.read_json('trial_task.json')

# 1)

In [4]:
def calculate_tariff(row):
    total_quantity = sum(prod['quantity'] for prod in row['products'])
    total_cost = (row['highway_cost'])
    delivery_tariff = total_cost / total_quantity
    return abs(delivery_tariff)

In [5]:
json_data['tariff'] = json_data.apply(calculate_tariff, axis=1)

all_name_tariff = json_data.loc[:, ['warehouse_name','tariff']]

task_1 = all_name_tariff.drop_duplicates(subset=['warehouse_name'])

### Ответ

In [6]:
task_1

,warehouse_name,tariff
0,Мордор,10.0
1,хутор близ Диканьки,15.0
2,отель Лето,25.0
5,остров невезения,5.0
6,гиперборея,20.0


# 2)

In [9]:
class Task2:
    def __init__(self, data):
        self.__products_list = []
        self.__quantity_list = []
        self.__income_list = []
        self.__expenses_list = []
        self.__profit_list = []

        self.__order_profit = []
        self.__order_id = []
        
        for index, row in data.iterrows():
            order_id = row['order_id']
            order_profit = 0
            
            for product in row['products']:
                product_name = product['product']
                product_quantity = product['quantity']
                product_income = product['quantity'] * product['price']
                product_expenses = product['quantity'] * row['tariff'] 
                product_profit = product_income - product_expenses
                
                self.__products_list.append(product_name)
                self.__quantity_list.append(product_quantity)
                self.__income_list.append(product_income)
                self.__expenses_list.append(product_expenses)
                self.__profit_list.append(product_profit)
                
                order_profit += product_profit
                      
            self.__order_id.append(order_id)
            self.__order_profit.append(order_profit)
            
        
    def __get_dataFrame_task2(self):
        result_df = pd.DataFrame({
            'product': self.__products_list,
            'quantity': self.__quantity_list,
            'income': self.__income_list,
            'expenses': self.__expenses_list,
            'profit': self.__profit_list
            })
        return result_df
    
    
    def get_result_task2(self):
        result_df_grouped = self.__get_dataFrame_task2().groupby('product').agg({
                                                            'quantity': 'sum',
                                                            'income': 'sum',
                                                            'expenses': 'sum',
                                                            'profit': 'sum'
                                                        }).reset_index()
        return result_df_grouped
    
    
    def __get_dataFrame_task3(self):
        print(len(self.__order_id))
        print(len(self.__order_profit))
        result_df = pd.DataFrame({
                            'order_id': self.__order_id,
                            'profit': self.__order_profit})
        return result_df
    
    
    def get_result_task3(self):
        return self.__get_dataFrame_task3()

In [10]:
obj3 = Task2(json_data)
task2 = obj3.get_result_task2()

### Ответ

In [11]:
task2

,product,quantity,income,expenses,profit
0,автограф Стаса Барецкого,48,28800,820.0,27980.0
1,билет в Израиль,58,58000,1050.0,56950.0
2,зеленая пластинка,61,610,920.0,-310.0
3,ломтик июльского неба,42,18900,670.0,18230.0
4,плюмбус,65,16250,940.0,15310.0
5,подписка на suppi-блог,33,4950,545.0,4405.0
6,статуэтка Ленина,68,13600,935.0,12665.0


# 3)

In [12]:
obj3 = Task2(json_data)
task3 = obj3.get_result_task3()

100
100


### Ответ

In [13]:
task3

,order_id,profit
0,11973,3980.0
1,62239,985.0
2,85794,-30.0
3,33684,1980.0
4,25824,975.0
...,...,...
95,28106,320.0
96,98100,1590.0
97,79293,1175.0
98,2930,720.0


In [14]:
task3['profit'].mean()

1352.3

# 4)


In [15]:
new_js_data =  json_data.merge(task3, how='inner')
new_js_data

,order_id,warehouse_name,highway_cost,products,tariff,profit
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':...",10.0,3980.0
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,...",15.0,985.0
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,...",25.0,-30.0
3,33684,Мордор,-30,"[{'product': 'билет в Израиль', 'price': 1000,...",10.0,1980.0
4,25824,отель Лето,-75,"[{'product': 'автограф Стаса Барецкого', 'pric...",25.0,975.0
...,...,...,...,...,...,...
95,28106,хутор близ Диканьки,-30,"[{'product': 'подписка на suppi-блог', 'price'...",15.0,320.0
96,98100,остров невезения,-10,"[{'product': 'автограф Стаса Барецкого', 'pric...",5.0,1590.0
97,79293,отель Лето,-75,"[{'product': 'статуэтка Ленина', 'price': 200,...",25.0,1175.0
98,2930,Мордор,-30,"[{'product': 'плюмбус', 'price': 250, 'quantit...",10.0,720.0


In [16]:
class Task4:
    def __init__(self, data):
        self.__warehouse_name_list = []
        self.__product_list = []
        self.__quantity_list = []
        self.__profit_list = []
        self.__product_of_warehouse_list = []
        
        for index, row in data.iterrows():
            warehouse_name = row['warehouse_name']
            warehouse_profit = data.loc[data['warehouse_name'] == warehouse_name, 'profit'].sum()
    
            for product in row['products']:
                name = product['product']
                price = product['price']
                quantity = product['quantity']
                tariff = row['tariff']
                
                income = price * quantity
                expenses = quantity * tariff
                profit = income - expenses
                
                product_of_warehouse = (profit / warehouse_profit) * 100
                
                self.__product_of_warehouse_list.append(product_of_warehouse)
                self.__warehouse_name_list.append(warehouse_name)
                self.__product_list.append(name)
                self.__quantity_list.append(quantity)
                self.__profit_list.append(profit)

                
    
    def get_result_task4(self):
        result = pd.DataFrame({
                    'warehouse_name': self.__warehouse_name_list,
                    'product': self.__product_list,
                    'quantity': self.__quantity_list,
                    'profit': self.__profit_list,
                    'percent_profit_product_of_warehouse': self.__product_of_warehouse_list
                })
        
        return result 

In [17]:
obj = Task4(new_js_data)
task4 = obj.get_result_task4()

### Ответ

In [18]:
task4

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse
0,Мордор,ломтик июльского неба,1,440.0,2.580645
1,Мордор,билет в Израиль,3,2970.0,17.419355
2,Мордор,статуэтка Ленина,3,570.0,3.343109
3,хутор близ Диканьки,билет в Израиль,1,985.0,2.645005
4,отель Лето,зеленая пластинка,2,-30.0,-0.110906
...,...,...,...,...,...
192,отель Лето,автограф Стаса Барецкого,1,575.0,2.125693
193,отель Лето,ломтик июльского неба,1,425.0,1.571165
194,Мордор,плюмбус,2,480.0,2.815249
195,Мордор,плюмбус,1,240.0,1.407625


# 5)

In [19]:
result_sorted = task4.sort_values(by='percent_profit_product_of_warehouse', ascending=False)
result_sorted

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse
1,Мордор,билет в Израиль,3,2970.0,17.419355
68,Мордор,билет в Израиль,2,1980.0,11.612903
5,Мордор,билет в Израиль,2,1980.0,11.612903
64,Мордор,билет в Израиль,2,1980.0,11.612903
136,отель Лето,билет в Израиль,3,2925.0,10.813309
...,...,...,...,...,...
161,отель Лето,зеленая пластинка,1,-15.0,-0.055453
26,гиперборея,зеленая пластинка,3,-30.0,-0.078370
174,отель Лето,зеленая пластинка,2,-30.0,-0.110906
4,отель Лето,зеленая пластинка,2,-30.0,-0.110906


In [20]:
result_sorted['accumulated_percent_profit_product_of_warehouse'] = result_sorted.groupby('warehouse_name')['percent_profit_product_of_warehouse'].cumsum()



### Ответ

In [21]:
result_sorted

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse
1,Мордор,билет в Израиль,3,2970.0,17.419355,17.419355
68,Мордор,билет в Израиль,2,1980.0,11.612903,29.032258
5,Мордор,билет в Израиль,2,1980.0,11.612903,40.645161
64,Мордор,билет в Израиль,2,1980.0,11.612903,52.258065
136,отель Лето,билет в Израиль,3,2925.0,10.813309,10.813309
...,...,...,...,...,...,...
161,отель Лето,зеленая пластинка,1,-15.0,-0.055453,100.388170
26,гиперборея,зеленая пластинка,3,-30.0,-0.078370,100.000000
174,отель Лето,зеленая пластинка,2,-30.0,-0.110906,100.277264
4,отель Лето,зеленая пластинка,2,-30.0,-0.110906,100.166359


# 6)

In [22]:
def assign_category(accumulated_percent):
    if accumulated_percent <= 70:
        return 'A'
    elif 70 < accumulated_percent <= 90:
        return 'B'
    else:
        return 'C'

In [23]:
result_sorted['category'] = result_sorted['accumulated_percent_profit_product_of_warehouse'].apply(assign_category)

### Ответ

In [24]:
result_sorted

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse,category
1,Мордор,билет в Израиль,3,2970.0,17.419355,17.419355,A
68,Мордор,билет в Израиль,2,1980.0,11.612903,29.032258,A
5,Мордор,билет в Израиль,2,1980.0,11.612903,40.645161,A
64,Мордор,билет в Израиль,2,1980.0,11.612903,52.258065,A
136,отель Лето,билет в Израиль,3,2925.0,10.813309,10.813309,A
...,...,...,...,...,...,...,...
161,отель Лето,зеленая пластинка,1,-15.0,-0.055453,100.388170,C
26,гиперборея,зеленая пластинка,3,-30.0,-0.078370,100.000000,C
174,отель Лето,зеленая пластинка,2,-30.0,-0.110906,100.277264,C
4,отель Лето,зеленая пластинка,2,-30.0,-0.110906,100.166359,C
